Import libraries

In [1]:
from ultralytics import YOLO
import tensorflow as tf
import numpy as np
import cv2
import os

In [2]:
LABELS = ["2-hands punch", "1-hand punch", "Standing", "Holding"] 
current_directory = os.getcwd()
project_path= os.path.abspath(os.path.join(current_directory, "../.."))

Load the model

In [4]:
LSTM_Model = tf.keras.models.load_model(f"{project_path}/model/best/model.keras")

Predict

In [ ]:
model = YOLO('yolov8n-pose.pt')

cap = cv2.VideoCapture(0)


steps = 0
lst_of_dct = {'key_1':[]}
label = " "
lst = []
predictions_lst = []
label_ = ''

while True:

    ret, image = cap.read()

    results = model.predict(source=image, conf=0.4, classes= 0, device=0, save=False)
    
    boxes = results[0].boxes
    result_keypoint = results[0].keypoints
   
    
    if len(result_keypoint[0].xy[0]) == 0:
        continue
    else:
        num = result_keypoint[0].xy[0].cpu().numpy()
        num = num.reshape((1, 34))
        
        lst_of_dct['key_1'].append(num)
        steps += 1
        
        if steps == 64:

            steps = 0
            lst=lst_of_dct['key_1']
            lst_of_dct = {'key_1':[]}
                
            lst = np.array(lst)
            lst = lst.reshape((1, 64, 34))
            
            preds = LSTM_Model.predict(lst)
            
            threshold = 0.8
            predicted_probabilities = preds[0]
            
            max_prob = np.max(predicted_probabilities)
        
            if max_prob > threshold:
                label = LABELS[np.argmax(preds)]
                predictions_lst.append(label)
            else:
                label = "NORMAL"
                predictions_lst.append(label)

        image = cv2.putText(image, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        image = cv2.resize(image, (1280, 720))
        cv2.imshow("Prediction", image)

        key = cv2.waitKey(1)
        if key == ord("a"):
            break


cv2.destroyAllWindows()